# Meshing

## Inviscid Wedge
![Inviscid Wedge](../assets/inviscid_wedge.png)

https://su2code.github.io/tutorials/Inviscid_Wedge/

# Method 1: Manually define lines and points

In [4]:
from typing import List
from ezmesh import CurveLoop, TransfinitePlaneSurface, Geometry, Point, Line, TransfiniteLine, TransfinitePlaneSurface, visualize_mesh


with Geometry() as geo:
    mesh_size=0.05
    points = [
        Point([0, 1], mesh_size),
        Point([1.5, 1], mesh_size),
        Point([1.5, 0.2], mesh_size),
        Point([0.5, 0], mesh_size),
        Point([0, 0], mesh_size),
    ]

    lines: List[Line] = [
        TransfiniteLine(start=points[0], end=points[1], label="upper", cell_count=150),
        TransfiniteLine(start=points[1], end=points[2], label="outlet", cell_count=200),
        TransfiniteLine(start=points[2], end=points[3], label="lower", cell_count=100),
        TransfiniteLine(start=points[3], end=points[4], label="lower", cell_count=50),
        TransfiniteLine(start=points[4], end=points[0], label="inlet", cell_count=200),
    ]

    wedge_curve_loop = CurveLoop(lines)
    surface = TransfinitePlaneSurface(
        outlines=[wedge_curve_loop], 
        is_quad_mesh=True, 
        corners=[]
    )
    mesh = geo.generate(surface)
    visualize_mesh(mesh)
    geo.write("mesh_wedge_inv.su2")


# %%


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 60%] Meshing curve 4 (Line)
Info    : [ 80%] Meshing curve 5 (Line)
Info    : Done meshing 1D (Wall 0.00759835s, CPU 0.005874s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 1.5168s, CPU 1.42546s)
Info    : Meshing 3D...
Info    : Done meshing 3D (Wall 3.59881e-05s, CPU 3e-05s)
Info    : 26726 nodes 53455 elements


HTML(value='Coords: ()')

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, far=1000.0, near=0.001, position=(0.0, 0.0, 1.0),…

,inlet
,lower
,outlet
,upper
,Zone 0


Info    : Writing 'mesh_wedge_inv.su2'...
Info    : Writing 52750 elements and 26726 nodes
Info    : Done writing 'mesh_wedge_inv.su2'


## Method 2: Automatically generate points and lines

In [2]:
from ezmesh import CurveLoop, PlaneSurface, Geometry, visualize_mesh
import numpy as np

with Geometry() as geo:
    wedge_coords = np.array([[0, 1], [1.5, 1], [1.5, 0.2], [0.5, 0], [0, 0]])
    wedge_curve_loop = CurveLoop(
        wedge_coords, 
        mesh_size=0.05,
        labels={
            "upper":  [0],
            "lower":  [2,3],
            "inlet":  [4],
            "outlet": [1],
        },
        transfinite_cell_counts={
            150: [0],
            200: [1,4],
            100: [2],
            50:  [3]
        }
    )
    surface = PlaneSurface(wedge_curve_loop, is_quad_mesh=True, transfinite_corners=[0,1,2,4])
    mesh = geo.generate(surface)
    visualize_mesh(mesh)
    geo.write("mesh_wedge_inv.su2")



Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 60%] Meshing curve 4 (Line)
Info    : [ 80%] Meshing curve 5 (Line)
Info    : Done meshing 1D (Wall 0.0020392s, CPU 0.001445s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Transfinite)
Info    : Done meshing 2D (Wall 0.00976s, CPU 0.009549s)
Info    : Meshing 3D...
Info    : Done meshing 3D (Wall 1.14189e-05s, CPU 1.2e-05s)
Info    : 30351 nodes 30705 elements


HTML(value='Coords: ()')

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, far=1000.0, near=0.001, position=(0.0, 0.0, 1.0),…

,upper
,inlet
,lower
,outlet
,Zone 0


Info    : Writing 'mesh_wedge_inv.su2'...
Info    : Writing 30000 elements and 30351 nodes
Info    : Done writing 'mesh_wedge_inv.su2'


## Import Existing Supported Meshes

In [7]:
from ezmesh import import_from_file
from ezmesh import visualize_mesh
mesh = import_from_file("mesh_wedge_inv.su2")
visualize_mesh(mesh)

HTML(value='Coords: ()')

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, far=1000.0, near=0.001, position=(0.0, 0.0, 1.0),…

,upper
,inlet
,lower
,outlet
,Zone 0


# Export SU2 Multi-Zone

In [8]:
from ezmesh.exporters import export_to_su2

export_to_su2([mesh,mesh], "mesh_wedge_inv_copy.su2")